In [0]:
# Configuração de credenciais e caminhos
from pyspark.sql.functions import col, lit, hour, to_date, year, month
from pyspark.sql.types import IntegerType, StringType



aws_access_key_id = "suacredencial"
aws_secret_access_key = "suacredencial"
s3_bucket_name = "ifood-case-data-lake-vitor"

silver_path = f"s3a://{s3_bucket_name}/silver_layer/"
gold_path = f"s3a://{s3_bucket_name}/gold_layer/"

read_options = {"fs.s3a.access.key": aws_access_key_id, "fs.s3a.secret.key": aws_secret_access_key}
write_options = {"fs.s3a.access.key": aws_access_key_id, "fs.s3a.secret.key": aws_secret_access_key}

In [0]:
# Leitura dos dados da camada Silver e unificação
df_silver_yellow = spark.read.options(**read_options).format("delta").load(f"{silver_path}yellow/")
df_silver_green = spark.read.options(**read_options).format("delta").load(f"{silver_path}green/")
df_silver_fhv = spark.read.options(**read_options).format("delta").load(f"{silver_path}fhv/")
df_silver_fhvhv = spark.read.options(**read_options).format("delta").load(f"{silver_path}fhvhv/")


# Unificação dos DataFrames para a camada Gold
df_gold_raw = (df_silver_yellow.unionByName(df_silver_green, allowMissingColumns=True)
                              .unionByName(df_silver_fhv, allowMissingColumns=True)
                              .unionByName(df_silver_fhvhv, allowMissingColumns=True))


In [0]:
# Criação das tabelas de dimensão (com chaves naturais)
# Dimensão de Vendor
df_vendor_dim = (df_gold_raw
    .select("vendor_id", "taxi_type")
    .distinct()
)
df_vendor_dim.write.options(**write_options).option("overwriteSchema", "true").format("delta").mode("overwrite").save(gold_path + "dim_vendor")
print("Tabela de dimensão 'dim_vendor' salva com sucesso.")

# Dimensão de Data
df_date_dim = (df_gold_raw
    .select(
        col("year").cast(IntegerType()),
        col("month").cast(IntegerType()),
        to_date(col("pickup_datetime")).alias("pickup_date")
    )
    .distinct()
)
df_date_dim.write.options(**write_options).option("overwriteSchema", "true").format("delta").mode("overwrite").save(gold_path + "dim_date")
print("Tabela de dimensão 'dim_date' salva com sucesso.")


Tabela de dimensão 'dim_vendor' salva com sucesso.
Tabela de dimensão 'dim_date' salva com sucesso.


In [0]:
# Criação da tabela de fatos
df_fact = (df_gold_raw
    .select(
        col("is_valid_total_amount"),
        col("is_valid_passenger_count"),
        col("is_valid_trip_time"),
        col("passenger_count"),
        col("total_amount"),
        col("pickup_datetime"),
        col("dropoff_datetime"),
        col("vendor_id"),
        col("taxi_type"),
        col("year"),
        col("month")
    )
)
(df_fact.write.options(**write_options)
    .option("overwriteSchema", "true")
    .format("delta")
    .mode("overwrite")
    .partitionBy("year", "month")
    .save(gold_path + "fact_trips")
)
print("Tabela de fatos 'fact_trips' salva com sucesso.")



Tabela de fatos 'fact_trips' salva com sucesso.


In [0]:

# Criacao de views temporárias para as consultas SQL
spark.read.options(**read_options).format("delta").load(f"{gold_path}fact_trips").createOrReplaceTempView("fact_trips")
spark.read.options(**read_options).format("delta").load(f"{gold_path}dim_vendor").createOrReplaceTempView("dim_vendor")
spark.read.options(**read_options).format("delta").load(f"{gold_path}dim_date").createOrReplaceTempView("dim_date")


In [0]:
# Análises solicitadas no case
# A primeira pergunta de análise é: "Qual a média de valor total (total_amount) recebido em um mês considerando todos os yellow táxis da frota?"
print("Média de valor total (total_amount) por mês para yellow táxis:")
spark.sql("""
    SELECT
        f.year,
        f.month,
        round(AVG(f.total_amount),2) AS media_valor_total_mensal
    FROM fact_trips AS f
    JOIN dim_vendor AS v ON f.vendor_id = v.vendor_id
    WHERE v.taxi_type = 'yellow'
    GROUP BY f.year, f.month
    ORDER BY f.year, f.month
""").show()

# A segunda pergunta de análise é: "Qual a média de passageiros (passenger_count) por cada hora do dia que pegaram táxi no mês de maio considerando todos os táxis da frota?"
print("Média de passageiros (passenger_count) por hora em maio para todos os táxis:")
spark.sql("""
    SELECT
        HOUR(f.pickup_datetime) AS hora_do_dia,
        round(AVG(f.passenger_count),2) AS media_passageiros
    FROM fact_trips AS f
    WHERE f.year = 2023 AND f.month = 5
    GROUP BY 1
    ORDER BY 1
""").show(50)

Média de valor total (total_amount) por mês para yellow táxis:
+----+-----+------------------------+
|year|month|media_valor_total_mensal|
+----+-----+------------------------+
|2023|    1|                   26.91|
|2023|    2|                   26.79|
|2023|    3|                   27.69|
|2023|    4|                   28.17|
|2023|    5|                   28.86|
+----+-----+------------------------+

Média de passageiros (passenger_count) por hora em maio para todos os táxis:
+-----------+-----------------+
|hora_do_dia|media_passageiros|
+-----------+-----------------+
|          0|             1.36|
|          1|             1.37|
|          2|             1.38|
|          3|             1.36|
|          4|             1.25|
|          5|             1.17|
|          6|             1.17|
|          7|             1.19|
|          8|              1.2|
|          9|             1.24|
|         10|             1.28|
|         11|              1.3|
|         12|             1.31|
|    